In [85]:
#Headers for the analysis
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option("mode.chained_assignment", None)
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
import skfuzzy as fuzz
import statsmodels.api as sm
import numpy as np
import pandas as pd
import math
from sklearn.metrics import mean_squared_error


In [ ]:
# All functions


In [158]:
#Filler functions
def rmse(predictions, targets):
    return np.sqrt(mean_squared_error(targets, predictions))#, squared=False)
# Fixed values for the device, not going to change based on our requirement

freq_dict = {}
freq_dict['gFreq'] = [177000000,266000000,350000000,420000000,480000000,600000000]
freq_dict['mFreq'] = [165000000,206000000,275000000,413000000,543000000,633000000,728000000,825000000]#,933000000]
freq_dict['sFreq'] = [200000,300000,400000,500000,600000,700000,800000,900000,1000000,1100000,1200000,1300000,1400000]
freq_dict['bFreq'] = [200000,300000,400000,500000,600000,700000,800000,900000,1000000,1100000,1200000,1300000,1400000,1500000,1600000,1700000,1800000,1900000,2000000]
# cost_list =['cCost','mCost','gCost']
cFreq = 'bFreq'
mFreq = 'mFreq'
gFreq = 'gFreq'

cost2FMap = {}
cost2FMap['cCost'] = 'bFreq'
cost2FMap['mCost'] = 'mFreq'
cost2FMap['gCost'] = 'gFreq'

## Function to extract the files and send a dataframe

def extractDatafromFile(fileName='./UtilTestFinal',padding=1): ## Generally make the padding 1
    new_data = pd.read_csv(fileName)
    #### This part needs to be 
    if padding == 0:  # if there is not 
        new_data['cCost'] =  new_data['bUtil']*new_data['bFreq']/2000000
        new_data['mCost'] = new_data['mUtil']*new_data['mFreq']/825000000
        new_data['gCost'] = 0
        new_data['gFreq'] = 600000000

    return new_data

#extractDatafromFile()
# Getting the exog list for analysis

def getExog(endog='cCost'):
    cost_list =['cCost','mCost','gCost']
    if endog not in cost_list:
        print("Incorrect Value")
        return None,None
    else:
        my_list = []
        for res in cost_list:
            if res != endog:
                my_list.append(res)
        return my_list[0],my_list[1]

# getExog('gCost')

## Function to shift the data, enabling testing and trainig

def createRequiredDataSet(input_df,endog,shiftAR,shiftARX):
    modified_df = pd.DataFrame()
    ## Get the exog values
    exog,exog1 = getExog(endog)
    modified_df = input_df[[endog,exog,exog1]]
    # Shifting the endog data
    for i in range(1,shiftAR+1):
        modified_df['%s_%d'%(endog,i)] = modified_df[endog].shift(i)

    # Shifting the exog and exog1 data
    modified_df[exog] = modified_df[exog].shift(1)
    modified_df[exog1] = modified_df[exog1].shift(1)

    for i in range(1,shiftARX+1):
        modified_df['%s_%d'%(exog,i)] = modified_df[exog].shift(i+1)
        modified_df['%s_%d'%(exog1,i)] = modified_df[exog1].shift(i+1)

    # Adding the frequency stuff
    modified_df[cFreq] = input_df[cFreq] 
    modified_df[mFreq] = input_df[mFreq] 
    modified_df[gFreq] = input_df[gFreq] 
    modified_df = modified_df.dropna()
    return modified_df


## Test_train divide

def test_train_split(input_df,trainIDX,testIDX,perc=0.8):
    totalLen = len(input_df)
    if trainIDX == -1 or testIDX == -1:
        trainIDX = int(perc*totalLen)
        testIDX = int(perc*totalLen)
    train_df = input_df.iloc[:trainIDX,:]
    test_df = input_df.iloc[testIDX:,:]
    return train_df,test_df


## get the matrices for training, we are assuming that 0 is the y value and the rest are x, we can choose to skip few columns

def getYXData(input_df,ignoreCols=0):
    totalLen = len(input_df.columns)
    y_data = input_df.iloc[:,0].to_numpy()
    x_data = input_df.iloc[:,1:totalLen-ignoreCols].to_numpy()
    return y_data,x_data


def train_Cluster(data, n_cluster = 5):
    # print("The stats of the training data: features {} and size {}".format(len(data[0]), len(data)))
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(data.T, n_cluster, 2, error=0.005, maxiter=1000, init=None, seed = 0)
    # print("The centroid values are \n {}".format(cntr))
    return cntr,u


def simple_train(yData, xData, weight=None):
    if weight is None:
        weight = np.ones(len(yData))

    new_x = sm.add_constant(xData)
    model = sm.WLS(yData,new_x,weights=weight)
    result = model.fit()
    return result



def simple_test_single(model, xData):
    newX = [1.0]
    newX.extend(xData)
    y_pred = model.get_prediction(newX).predicted_mean
    return y_pred

def getSingleCluster(data, cntr):
    ut,u0t,dt,jmt,pt,fpct = fuzz.cluster.cmeans_predict(np.array([data]).T,cntr,2,error=0.005,maxiter=1000,init=None,seed=0)
    return ut


def prediction(model,x,cluster):
    ## get cluster
    N_CLUST = len(model)
    prob_d = getSingleCluster(x,cluster).T[0]
    y = []
    for i in range(N_CLUST):
        ys = simple_test_single(model[i],x)
        y.append(ys*prob_d[i])
    my_pred = min(sum(y)[0],100.0)
    return my_pred

In [162]:


# my_test = extractDatafromFile('./UtilTestFinal',0)
# Mod = createRequiredDataSet(my_test,'cCost',5,3)   
    
    

In [178]:
N_CLUST=5
TEST_LEN = 50
endog = 'mCost'
my_test = extractDatafromFile('./UtilTestFinal',0)
Mod = createRequiredDataSet(my_test,endog='cCost',shiftAR=5,shiftARX=3)
train_df,test_df = test_train_split(Mod,trainIDX=5000,testIDX=5000,perc=0.0)
y_train,x_train = getYXData(train_df,ignoreCols=3)
y_test,x_test = getYXData(test_df,ignoreCols=3)
### Get the clusters
cluster_center,prob = train_Cluster(x_train[:,:],n_cluster = N_CLUST)
### Train the models
model = []
for i in range(N_CLUST):
    out_model = simple_train(y_train,x_train,weight = prob[i,:])
    model.append(out_model)

## Predict the values
ys_target = []
for id in range(TEST_LEN):
    predVal = prediction(model,x_test[id],cluster_center)
    ys_target.append(predVal)

ys_freq = test_df[cost2FMap[endog]].to_numpy()[:TEST_LEN]
ys_freq

array([206000000, 275000000, 165000000, 165000000, 413000000, 165000000,
       165000000, 165000000, 275000000, 206000000, 206000000, 275000000,
       275000000, 206000000, 275000000, 165000000, 165000000, 413000000,
       165000000, 275000000, 275000000, 206000000, 275000000, 275000000,
       275000000, 206000000, 206000000, 275000000, 206000000, 206000000,
       275000000, 275000000, 206000000, 206000000, 275000000, 275000000,
       206000000, 206000000, 275000000, 275000000, 543000000, 165000000,
       165000000, 413000000, 413000000, 165000000, 275000000, 206000000,
       206000000, 275000000])

In [171]:
def ModelPipeline(fileName,endog='cCost',N_CLUST=5,shiftAR=5,shiftARX=3,trainIDX=5000,testIDX=5000,TEST_LEN = 50, energy=True):
    my_test = extractDatafromFile(fileName,0) # set to 1 for most of the data, unless no GPU present
    Mod = createRequiredDataSet(my_test,endog,shiftAR,shiftARX)
    train_df,test_df = test_train_split(Mod,trainIDX,testIDX,perc=0.0)
    y_train,x_train = getYXData(train_df,ignoreCols=3)
    y_test,x_test = getYXData(test_df,ignoreCols=3)
    ### Get the clusters
    cluster_center,prob = train_Cluster(x_train[:,:],n_cluster = N_CLUST)
    ### Train the Model
    model = []
    for i in range(N_CLUST):
        out_model = simple_train(y_train,x_train,weight = prob[i,:])
        model.append(out_model)

    ### Test the models
    ys_actual = y_test[:TEST_LEN]
    ys_est = []
    for id in range(TEST_LEN):
        predVal = prediction(model,x_test[id],cluster_center)
        ys_est.append(predVal)
    # print(cost2FMap[endog])        
    if energy:
        ys_freq = test_df[cost2FMap[endog]].to_numpy()[:TEST_LEN] # the actual frequency we observed
        
    ##
    
    return ys_est,ys_actual,rmse(ys_est,ys_actual)

In [172]:
y_est,y_act,err = ModelPipeline('./UtilTestFinal','mCost',N_CLUST=3,shiftAR=20,shiftARX=3,trainIDX=5000,testIDX=5000,TEST_LEN = 200)

In [173]:
err

0.3926820928647231

In [193]:
for ar in range(3,20):
    y_est,y_act,err = ModelPipeline('./UtilTestFinal','cCost',N_CLUST=10,shiftAR=ar,shiftARX=ar-2,trainIDX=7000,testIDX=3000,TEST_LEN = 500)
    print("#AR ",ar," RMSE :", err)
    

#AR  3  RMSE : 16.36270071265466
#AR  4  RMSE : 15.906601209119556
#AR  5  RMSE : 15.618240552014347
#AR  6  RMSE : 15.541488693273728
#AR  7  RMSE : 15.398540020277032
#AR  8  RMSE : 15.378594183559889
#AR  9  RMSE : 15.371845259005767
#AR  10  RMSE : 15.36300572546293
#AR  11  RMSE : 15.36512330063981
#AR  12  RMSE : 15.585806468788252
#AR  13  RMSE : 15.564586188566665
#AR  14  RMSE : 15.702221794697364
#AR  15  RMSE : 15.72287160818384
#AR  16  RMSE : 15.754728981851745
#AR  17  RMSE : 15.743832026481673
#AR  18  RMSE : 15.663522646487932
#AR  19  RMSE : 15.612837402410577
